**Upper Bound (QP) - 2D**

In [ ]:
import cvxpy as cp
import numpy as np
from typing import List

def g1_ub_qp_2D(seen: List[List[int]], row_sum: List[int], col_sum: List[int]) -> int:
    n = len(row_sum)
    m = len(col_sum)
    seen_array = np.array(seen)

    # optimization variables
    matrix = cp.Variable((n, m), nonneg=True)

    # precomputed row and column totals of the seen matrix
    seen_row_sums = seen_array.sum(axis=1)
    seen_col_sums = seen_array.sum(axis=0)

    # objective: minimize sum of squares of seen + matrix
    objective = cp.Minimize(cp.sum_squares(seen_array + matrix))

    # constraints for row and column sums
    constraints = [
        cp.sum(matrix, axis=1) + seen_row_sums <= row_sum,
        cp.sum(matrix, axis=0) + seen_col_sums <= col_sum
    ]

    # define solver
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.SCS)

    if problem.status != cp.OPTIMAL:
        raise ValueError("Optimization was not successful")

    g1ErrorUB = problem.value
    row_square = np.dot(row_sum, row_sum)
    conflict = (row_square - g1ErrorUB) / 2
    return conflict

**Lower Bound (Ignore Row Sum) - 2D**

In [ ]:
import numpy as np
from typing import List

def g1_lb_ignoreSum_2D(seen: List[List[int]], row_sum: List[int], col_sum: List[int]) -> int:
  row_sum_cur = []
  row_max_index = []
  remaining_row_sum = []

  n = len(row_sum)
  m = len(col_sum)

  # Calculate conflict
  xi_square = 0
  for i in range(n):
    row_sum_cur.append(0)
    row_max_index.append(0)
    remaining_row_sum.append(0)
    for j in range(m):
      row_sum_cur[i] += seen[i][j]
      if seen[i][row_max_index[i]] < seen[i][j]:
        row_max_index[i] = j
      xi_square += seen[i][j]*seen[i][j]
    remaining_row_sum[i] = row_sum[i] - row_sum_cur[i]
    _max = seen[i][row_max_index[i]]
    xi_square += remaining_row_sum[i] * remaining_row_sum[i] + 2 * _max * remaining_row_sum[i]
  row_square = 0
  for r_s in row_sum:
    row_square += r_s*r_s
  conflict = (row_square - xi_square)/2
  return conflict

**Bounds (QP) - 3D**

In [ ]:
!pip install gurobipy

In [ ]:
import gurobipy as gp
import numpy as np
from typing import List

def g1_bounds_qp_3D(
    seen: List[List[List[int]]],
    row_sum_1: List[int],
    row_sum_2: List[int],
    col_sum: List[int],
    lb_calc: bool
) -> int:

    n1 = len(row_sum_1)
    n2 = len(row_sum_2)
    m = len(col_sum)

    seen_np = np.array(seen)

    model = gp.Model()
    model.setParam("OutputFlag", 0)

    # non-negative decision variables
    matrix_vars = model.addVars(n1, n2, m, lb=0, name="matrix")

    # objective function: sum over (seen + x)[i,j,k1] * (seen + x)[i,j,k2] for k1 < k2
    objective_terms = []
    for i in range(n1):
        for j in range(n2):
            for k1 in range(m - 1):
                for k2 in range(k1 + 1, m):
                    t1 = seen_np[i, j, k1] + matrix_vars[i, j, k1]
                    t2 = seen_np[i, j, k2] + matrix_vars[i, j, k2]
                    objective_terms.append(t1 * t2)

    model.setObjective(gp.quicksum(objective_terms), gp.GRB.MINIMIZE if lb_calc else gp.GRB.MAXIMIZE)

    # row sum constraints
    for i in range(n1):
        rhs = row_sum_1[i] - seen_np[i, :, :].sum()
        model.addConstr(
            gp.quicksum(matrix_vars[i, j, k] for j in range(n2) for k in range(m)) <= rhs,
            name=f"RowSum1_{i}"
        )

    for j in range(n2):
        rhs = row_sum_2[j] - seen_np[:, j, :].sum()
        model.addConstr(
            gp.quicksum(matrix_vars[i, j, k] for i in range(n1) for k in range(m)) <= rhs,
            name=f"RowSum2_{j}"
        )

    # col sum constraints
    for k in range(m):
        rhs = col_sum[k] - seen_np[:, :, k].sum()
        model.addConstr(
            gp.quicksum(matrix_vars[i, j, k] for i in range(n1) for j in range(n2)) <= rhs,
            name=f"ColSum_{k}"
        )

    model.optimize()

    if model.status == gp.GRB.OPTIMAL:
        return model.objVal
    else:
        raise ValueError("No optimal solution found.")

**Bounds (QP) - 4D**

In [ ]:
import gurobipy as gp
import numpy as np
from typing import List

def g1_bounds_qp_4D(
    seen: List[List[List[List[int]]]],
    row_sum_1: List[int],
    row_sum_2: List[int],
    row_sum_3: List[int],
    col_sum: List[int],
    lb_calc: bool
) -> int:

    n1 = len(row_sum_1)
    n2 = len(row_sum_2)
    n3 = len(row_sum_3)
    m = len(col_sum)

    seen_np = np.array(seen)

    model = gp.Model()
    model.setParam("OutputFlag", 0)

    # decision variables: non-negative
    matrix_vars = model.addVars(n1, n2, n3, m, lb=0, name="matrix")

    # objective: sum over (seen + matrix)[i,j,k,l1] * (seen + matrix)[i,j,k,l2] for l1 < l2
    objective_terms = []
    for i in range(n1):
        for j in range(n2):
            for k in range(n3):
                for l1 in range(m - 1):
                    for l2 in range(l1 + 1, m):
                        term1 = seen_np[i, j, k, l1] + matrix_vars[i, j, k, l1]
                        term2 = seen_np[i, j, k, l2] + matrix_vars[i, j, k, l2]
                        objective_terms.append(term1 * term2)

    if lb_calc:
        model.setObjective(gp.quicksum(objective_terms), gp.GRB.MINIMIZE)
    else:
        model.setObjective(gp.quicksum(objective_terms), gp.GRB.MAXIMIZE)

    # Constraints

    # 1D row sums
    for i in range(n1):
        lhs = gp.quicksum(matrix_vars[i, j, k, l] for j in range(n2) for k in range(n3) for l in range(m))
        rhs = row_sum_1[i] - seen_np[i, :, :, :].sum()
        model.addConstr(lhs <= rhs, name=f"RowSum1_{i}")

    # 2D row sums
    for j in range(n2):
        lhs = gp.quicksum(matrix_vars[i, j, k, l] for i in range(n1) for k in range(n3) for l in range(m))
        rhs = row_sum_2[j] - seen_np[:, j, :, :].sum()
        model.addConstr(lhs <= rhs, name=f"RowSum2_{j}")

    # 3D row sums
    for k in range(n3):
        lhs = gp.quicksum(matrix_vars[i, j, k, l] for i in range(n1) for j in range(n2) for l in range(m))
        rhs = row_sum_3[k] - seen_np[:, :, k, :].sum()
        model.addConstr(lhs <= rhs, name=f"RowSum3_{k}")

    # Column sums
    for l in range(m):
        lhs = gp.quicksum(matrix_vars[i, j, k, l] for i in range(n1) for j in range(n2) for k in range(n3))
        rhs = col_sum[l] - seen_np[:, :, :, l].sum()
        model.addConstr(lhs <= rhs, name=f"ColSum_{l}")

    model.optimize()

    if model.status == gp.GRB.OPTIMAL:
        return model.objVal
    else:
        raise ValueError("No optimal solution found.")

**Approx Top-K Ranking (Sampling/Mean)**

In [ ]:
import random
from collections import defaultdict

# Approach 1 : Using mean to find top-k
def cal_mean(attributes, ubDict, lbDict, e):

  meanTopk = []
  emax_attr = []

  for attribute in attributes:
    lowerB = lbDict[attribute]
    upperB = ubDict[attribute]

    if upperB <= e:
        emax_attr.append(attribute)

  for attr in emax_attr:
    lb = lbDict[attr]
    ub = ubDict[attr]
    meanVal = (lb+ub)/2
    meanTopk.append((attr, meanVal))

  meanTopk = sorted(meanTopk, key=lambda x: x[1])

  meanTopk = [attr[0] for attr in meanTopk]

  return meanTopk

# Approach 2 : Using sampling to find top-k
def cal_sample_prob(attributes, ubDict, lbDict, e):

  emax_attr = []
  order_counts = defaultdict(int)

  for attribute in attributes:
    lowerB = lbDict[attribute]
    upperB = ubDict[attribute]

    if upperB <= e:
      emax_attr.append(attribute)

  for _ in range(1000):
    samples = {}
    for attr in emax_attr:
      lower_bound = lbDict[attr]
      upper_bound = ubDict[attr]
      samples[str(attr)] = random.uniform(lower_bound, upper_bound)

    sorted_order = tuple(sorted(samples, key=samples.get))
    order_counts[sorted_order] += 1

  most_frequent_order = max(order_counts, key=order_counts.get)

  return most_frequent_order

**Actual Top-K**

In [ ]:
import numpy as np
import pandas as pd
import itertools
from itertools import combinations

def calculateActualTopk(file_name, e):

  mainDF = pd.read_csv(file_name, header = 0)
  # last column in dataset is the Z attribute
  A_name_list = mainDF.columns[:-1].tolist()

  numOfRows = len(mainDF)
  norm = numOfRows*(numOfRows-1)

  # THREE-LHS AFDs

  actualConflictDict_4D = {}

  attribute_triplets = list(combinations(mainDF.columns[:-1], 3))

  for attr1, attr2, attr3 in attribute_triplets:

    A_name = attr1+attr2+attr3

    attr1_values = mainDF[attr1].unique().tolist()
    attr2_values = mainDF[attr2].unique().tolist()
    attr3_values = mainDF[attr3].unique().tolist()
    target_values = mainDF['y'].unique().tolist()

    matrix_3lhs = [[[[0 for _ in target_values] for _ in attr3_values] for _ in attr2_values] for _ in attr1_values]

    attr1_indices_map = {value: idx for idx, value in enumerate(attr1_values)}
    attr2_indices_map = {value: idx for idx, value in enumerate(attr2_values)}
    attr3_indices_map = {value: idx for idx, value in enumerate(attr3_values)}
    target_indices_map = {value: idx for idx, value in enumerate(target_values)}

    rowSum_a1 = [0]*len(attr1_indices_map)
    rowSum_a2 = [0]*len(attr2_indices_map)
    rowSum_a3 = [0]*len(attr3_indices_map)
    colSum = [0]*len(target_indices_map)

    for index, row in mainDF.iterrows():
      a1_index = attr1_indices_map[row[attr1]]
      a2_index = attr2_indices_map[row[attr2]]
      a3_index = attr3_indices_map[row[attr3]]
      z_index = target_indices_map[row['y']]

      rowSum_a1[a1_index] += 1
      rowSum_a2[a2_index] += 1
      rowSum_a3[a3_index] += 1
      colSum[z_index] += 1

    for index, row in mainDF.iterrows():

      val1 = row[attr1]
      val2 = row[attr2]
      val3 = row[attr3]
      target_val = row['y']

      if val1 in attr1_indices_map and val2 in attr2_indices_map and val3 in attr3_indices_map and target_val in target_indices_map:

        row_index = attr1_indices_map[val1]
        col_index = attr2_indices_map[val2]
        depth_index = attr3_indices_map[val3]
        target_index = target_indices_map[target_val]

        matrix_3lhs[row_index][col_index][depth_index][target_index] += 1

    g1Error_4D = 0

    for matrix_3d in matrix_3lhs:
      for matrix_2d in matrix_3d:
          for row in matrix_2d:
              pairs = itertools.combinations(row, 2)
              for pair in pairs:
                  g1Error_4D += pair[0] * pair[1]

    actualConflictDict_4D[str(A_name)] = g1Error_4D/norm

  # THREE-LHS AFDs

  # TWO-LHS AFDs

  actualConflictDict_3D = {}

  attribute_pairs = list(combinations(mainDF.columns[:-1], 2))

  for attr1, attr2 in attribute_pairs:

    A_name = attr1+attr2
    attr1_values = mainDF[attr1].unique().tolist()
    attr2_values = mainDF[attr2].unique().tolist()
    target_values = mainDF['y'].unique().tolist()

    matrix_2lhs = [[[0 for _ in target_values] for _ in attr2_values] for _ in attr1_values]

    attr1_indices_map = {value: idx for idx, value in enumerate(attr1_values)}
    attr2_indices_map = {value: idx for idx, value in enumerate(attr2_values)}
    target_indices_map = {value: idx for idx, value in enumerate(target_values)}

    rowSum_a1 = [0]*len(attr1_indices_map)
    rowSum_a2 = [0]*len(attr2_indices_map)
    colSum = [0]*len(target_indices_map)

    for index, row in mainDF.iterrows():
      a1_index = attr1_indices_map[row[attr1]]
      a2_index = attr2_indices_map[row[attr2]]
      z_index = target_indices_map[row['y']]

      rowSum_a1[a1_index] += 1
      rowSum_a2[a2_index] += 1
      colSum[z_index] += 1

    for index, row in mainDF.iterrows():

      val1 = row[attr1]
      val2 = row[attr2]
      target_val = row['y']

      if val1 in attr1_indices_map and val2 in attr2_indices_map and target_val in target_indices_map:

        row_index = attr1_indices_map[val1]
        col_index = attr2_indices_map[val2]
        target_index = target_indices_map[target_val]

        matrix_2lhs[row_index][col_index][target_index] += 1

    g1Error_3D = 0

    for matrix_2d in matrix_2lhs:
      for row in matrix_2d:
          pairs = itertools.combinations(row, 2)
          for pair in pairs:
              g1Error_3D += pair[0] * pair[1]

    actualConflictDict_3D[str(A_name)] = g1Error_3D/norm

  # TWO-LHS AFDs

  # ONE-LHS AFDs

  actualConflictDict_2D = {}

  for A_name in A_name_list:

    Z_name = "y"
    df = mainDF[[A_name, Z_name]]

    a_indices = np.unique(df[A_name])
    a_indices = a_indices.tolist()
    a_indices_map = dict(zip(a_indices, range(len(a_indices))))

    z_indices = np.unique(df[Z_name])
    z_indices = z_indices.tolist()
    z_indices_map = dict(zip(z_indices, range(len(z_indices))))

    row_sum = [0]*len(a_indices)
    col_sum = [0]*len(z_indices)

    matrix = []
    for i in a_indices:
      matrix.append([0]*len(z_indices))

    dataset = df.values.tolist()

    for data in dataset:
      row_index = a_indices_map[data[0]]
      col_index = z_indices_map[data[1]]
      row_sum[row_index] += 1
      col_sum[col_index] += 1
      matrix[row_index][col_index] += 1

    g1Error_2D = 0
    for row in matrix:
      pairs = itertools.combinations(row, 2)
      for pair in pairs:
        g1Error_2D += pair[0] * pair[1]

    actualConflictDict_2D[A_name] = g1Error_2D/norm

  # ONE-LHS AFDs

  mergedConflictDict = {**actualConflictDict_2D, **actualConflictDict_3D, **actualConflictDict_4D}
  unsortedConflictDict = {k: v for k, v in mergedConflictDict.items() if v <= e}
  sortedConflictDict = {k: v for k, v in sorted(mergedConflictDict.items(), key=lambda item: item[1]) if v <= e}
  actualTopk = sorted(sortedConflictDict, key=lambda k: sortedConflictDict[k])

  return sortedConflictDict, unsortedConflictDict

**LPDS - Uninformed Prior**

In [ ]:
import os
import time
import numpy as np
import pandas as pd

def removeSingleMarginals(calc_matrix, row_sum, col_sum):

  row_sum_copy = row_sum[:]
  col_sum_copy = col_sum[:]
  matrix_copy = [row[:] for row in calc_matrix]

  rows_to_remove = [i for i, rsum in enumerate(row_sum_copy) if rsum == 1]

  for i in sorted(rows_to_remove, reverse=True):
        row = matrix_copy[i]
        for j, val in enumerate(row):
            col_sum_copy[j] -= val

        del matrix_copy[i]
        del row_sum_copy[i]

  return matrix_copy, row_sum_copy, col_sum_copy

def calculateBoundsLPDS(file_name, dataInterval, stopAtIndex):

  mainDF = pd.read_csv(file_name, header = 0)

  numOfRows = len(mainDF)
  norm = numOfRows*(numOfRows-1)

  # last column in dataset is the Z attribute
  A_name_list = mainDF.columns[:-1].tolist()

  mainUBDict = {}
  mainLBDict = {}

  boundsTime2D = {}
  boundsTime3D = {}
  boundsTime4D = {}

  ub2TimeDict = {}
  ub3TimeDict = {}
  ub4TimeDict = {}
  lb2TimeDict = {}
  lb3TimeDict = {}
  lb4TimeDict = {}

  # THREE-LHS AFDs

  attribute_triplets = list(combinations(mainDF.columns[:-1], 3))

  for attr1, attr2, attr3 in attribute_triplets:

    A_name = attr1+attr2+attr3

    attr1_values = mainDF[attr1].unique().tolist()
    attr2_values = mainDF[attr2].unique().tolist()
    attr3_values = mainDF[attr3].unique().tolist()
    target_values = mainDF['y'].unique().tolist()

    op1 = []

    matrix_3lhs = [[[[0 for _ in target_values] for _ in attr3_values] for _ in attr2_values] for _ in attr1_values]

    attr1_indices_map = {value: idx for idx, value in enumerate(attr1_values)}
    attr2_indices_map = {value: idx for idx, value in enumerate(attr2_values)}
    attr3_indices_map = {value: idx for idx, value in enumerate(attr3_values)}
    target_indices_map = {value: idx for idx, value in enumerate(target_values)}

    rowSum_a1 = [0]*len(attr1_indices_map)
    rowSum_a2 = [0]*len(attr2_indices_map)
    rowSum_a3 = [0]*len(attr3_indices_map)
    colSum = [0]*len(target_indices_map)

    for index, row in mainDF.iterrows():
      a1_index = attr1_indices_map[row[attr1]]
      a2_index = attr2_indices_map[row[attr2]]
      a3_index = attr3_indices_map[row[attr3]]
      z_index = target_indices_map[row['y']]

      rowSum_a1[a1_index] += 1
      rowSum_a2[a2_index] += 1
      rowSum_a3[a3_index] += 1
      colSum[z_index] += 1

    for index, row in mainDF.iterrows():

      if index > stopAtIndex:
        break

      val1 = row[attr1]
      val2 = row[attr2]
      val3 = row[attr3]
      target_val = row['y']

      if val1 in attr1_indices_map and val2 in attr2_indices_map and val3 in attr3_indices_map and target_val in target_indices_map:

        row_index = attr1_indices_map[val1]
        col_index = attr2_indices_map[val2]
        depth_index = attr3_indices_map[val3]
        target_index = target_indices_map[target_val]

        matrix_3lhs[row_index][col_index][depth_index][target_index] += 1

        if index % dataInterval == 0 or index == len(mainDF)-1:

          if index not in mainUBDict:
            mainUBDict[index] = {}

          if index not in mainLBDict:
            mainLBDict[index] = {}

          startTime4D = time.time()

          startUBTime4D = time.time()
          ubValue = g1_bounds_qp_4D(matrix_3lhs, rowSum_a1, rowSum_a2, rowSum_a3, colSum, False)
          stopUBTime4D = time.time()

          startLBTime4D = time.time()
          lbValue = g1_bounds_qp_4D(matrix_3lhs, rowSum_a1, rowSum_a2, rowSum_a3, colSum, True)
          stopLBTime4D = time.time()

          stopTime4D = time.time()

          boundsTime4D[index] = stopTime4D - startTime4D

          ub4TimeDict[index] = stopUBTime4D - startUBTime4D
          lb4TimeDict[index] = stopLBTime4D - startLBTime4D

          ubValue = ubValue/norm
          lbValue = lbValue/norm

          if not op1:
            op1.append([index+1, ubValue, lbValue])
            u1 = ubValue
            l2 = lbValue
          else:
            op1.append([index+1, min(ubValue, op1[-1][1]), max(lbValue, op1[-1][2])])
            u1 = min(ubValue, op1[-1][1])
            l2 = max(lbValue, op1[-1][2])

          mainUBDict[index][A_name] = u1
          mainLBDict[index][A_name] = l2

  # THREE-LHS AFDs

  # TWO-LHS AFDs

  attribute_pairs = list(combinations(mainDF.columns[:-1], 2))

  for attr1, attr2 in attribute_pairs:

    A_name = attr1+attr2
    attr1_values = mainDF[attr1].unique().tolist()
    attr2_values = mainDF[attr2].unique().tolist()
    target_values = mainDF['y'].unique().tolist()

    op1 = []

    matrix_2lhs = [[[0 for _ in target_values] for _ in attr2_values] for _ in attr1_values]

    attr1_indices_map = {value: idx for idx, value in enumerate(attr1_values)}
    attr2_indices_map = {value: idx for idx, value in enumerate(attr2_values)}
    target_indices_map = {value: idx for idx, value in enumerate(target_values)}

    rowSum_a1 = [0]*len(attr1_indices_map)
    rowSum_a2 = [0]*len(attr2_indices_map)
    colSum = [0]*len(target_indices_map)

    for index, row in mainDF.iterrows():
      a1_index = attr1_indices_map[row[attr1]]
      a2_index = attr2_indices_map[row[attr2]]
      z_index = target_indices_map[row['y']]

      rowSum_a1[a1_index] += 1
      rowSum_a2[a2_index] += 1
      colSum[z_index] += 1

    for index, row in mainDF.iterrows():

      if index > stopAtIndex:
        break

      val1 = row[attr1]
      val2 = row[attr2]
      target_val = row['y']

      if val1 in attr1_indices_map and val2 in attr2_indices_map and target_val in target_indices_map:

        row_index = attr1_indices_map[val1]
        col_index = attr2_indices_map[val2]
        target_index = target_indices_map[target_val]

        matrix_2lhs[row_index][col_index][target_index] += 1

        if index % dataInterval == 0 or index == len(mainDF)-1:

          startTime3D = time.time()

          startUBTime3D = time.time()
          ubValue = g1_bounds_qp_3D(matrix_2lhs, rowSum_a1, rowSum_a2, colSum, False)
          stopUBTime3D = time.time()

          startLBTime3D = time.time()
          lbValue = g1_bounds_qp_3D(matrix_2lhs, rowSum_a1, rowSum_a2, colSum, True)
          stopLBTime3D = time.time()

          stopTime3D = time.time()

          boundsTime3D[index] = stopTime3D - startTime3D

          ub3TimeDict[index] = stopUBTime3D - startUBTime3D
          lb3TimeDict[index] = stopLBTime3D - startLBTime3D

          ubValue = ubValue/norm
          lbValue = lbValue/norm

          if not op1:
            op1.append([index+1, ubValue, lbValue])
            u1 = ubValue
            l2 = lbValue
          else:
            op1.append([index+1, min(ubValue, op1[-1][1]), max(lbValue, op1[-1][2])])
            u1 = min(ubValue, op1[-1][1])
            l2 = max(lbValue, op1[-1][2])

          mainUBDict[index][A_name] = u1
          mainLBDict[index][A_name] = l2

  # TWO-LHS AFDs

  # ONE-LHS AFDs

  for A_name in A_name_list:

    Z_name = "y"
    df = mainDF[[A_name, Z_name]]

    a_indices = np.unique(df[A_name])
    a_indices = a_indices.tolist()
    a_indices_map = dict(zip(a_indices, range(len(a_indices))))

    z_indices = np.unique(df[Z_name])
    z_indices = z_indices.tolist()
    z_indices_map = dict(zip(z_indices, range(len(z_indices))))

    dataset = df.values.tolist()
    row_sum = [0]*len(a_indices)
    col_sum = [0]*len(z_indices)

    for data in dataset:
      row_index = a_indices_map[data[0]]
      col_index = z_indices_map[data[1]]
      row_sum[row_index] += 1
      col_sum[col_index] += 1

    op1 = []
    calc_matrix = []

    for i in a_indices:
      calc_matrix.append([0]*len(z_indices))

    for index,data in enumerate(dataset):

      if index > stopAtIndex:
        break

      row_index = a_indices_map[data[0]]
      col_index = z_indices_map[data[1]]
      calc_matrix[row_index][col_index] += 1

      if index % dataInterval == 0 or index == len(dataset)-1:

        mat, rowS, colS = removeSingleMarginals(calc_matrix, row_sum, col_sum)

        startTime2D = time.time()

        startUBTime2D = time.time()
        ubValue = g1_ub_qp_2D(mat, rowS, colS)
        stopUBTime2D = time.time()

        startLBTime2D = time.time()
        lbValue = g1_lb_ignoreSum_2D(mat, rowS, colS)
        stopLBTime2D = time.time()

        stopTime2D = time.time()

        ub2TimeDict[index] = stopUBTime2D - startUBTime2D
        lb2TimeDict[index] = stopLBTime2D - startLBTime2D

        boundsTime2D[index] = stopTime2D - startTime2D

        ubValue = ubValue/norm
        lbValue = lbValue/norm

        if not op1:
          op1.append([index+1, ubValue, lbValue])
          u1 = ubValue
          l1 = lbValue

        else:
          op1.append([index+1, min(ubValue, op1[-1][1]), max(lbValue, op1[-1][2])])
          u1 = min(ubValue, op1[-1][1])
          l1 = max(lbValue, op1[-1][2])

        mainUBDict[index][A_name] = u1
        mainLBDict[index][A_name] = l1

  # ONE-LHS AFDs

  indexTotalTimeDict = {key: boundsTime2D.get(key, 0) + boundsTime3D.get(key, 0) + boundsTime4D.get(key, 0)
               for key in set(boundsTime2D) | set(boundsTime3D) | set(boundsTime4D)}

  indexTotalUBTimeDict = {key: ub2TimeDict.get(key, 0) + ub3TimeDict.get(key, 0) + ub4TimeDict.get(key, 0)
               for key in set(ub2TimeDict) | set(ub3TimeDict) | set(ub4TimeDict)}

  indexTotalLBTimeDict = {key: lb2TimeDict.get(key, 0) + lb3TimeDict.get(key, 0) + lb4TimeDict.get(key, 0)
               for key in set(lb2TimeDict) | set(lb3TimeDict) | set(lb4TimeDict)}

  df1 = pd.DataFrame(list(boundsTime2D.items()), columns=['Index', 'Time'])
  df2 = pd.DataFrame(list(boundsTime3D.items()), columns=['Index', 'Time'])
  df3 = pd.DataFrame(list(boundsTime4D.items()), columns=['Index', 'Time'])
  df4 = pd.DataFrame(list(indexTotalTimeDict.items()), columns=['Index', 'Time'])
  df5 = pd.DataFrame(list(indexTotalUBTimeDict.items()), columns=['Index', 'Time'])
  df6 = pd.DataFrame(list(indexTotalLBTimeDict.items()), columns=['Index', 'Time'])

  df1.to_excel('2DTime.xlsx', index=False)
  df2.to_excel('3DTime.xlsx', index=False)
  df3.to_excel('4DTime.xlsx', index=False)
  df4.to_excel('TotalTimeLP.xlsx', index=False)
  df5.to_excel('AblationStudyUB.xlsx', index=False)
  df6.to_excel('AblationStudyLB.xlsx', index=False)

  return mainUBDict, mainLBDict, indexTotalTimeDict, boundsTime2D, boundsTime3D, boundsTime4D

**IPF - Informed Prior**

In [ ]:
!pip install ipfn
import numpy as np
import pandas as pd
from ipfn import ipfn

def calError_2D(matrix):
  error = 0
  for row in matrix:
    pairs = itertools.combinations(row, 2)
    for pair in pairs:
      error += pair[0] * pair[1]
  return error

def calError_3D(matrix):
  error = 0
  for matrix_2d in matrix:
    for row in matrix_2d:
      pairs = itertools.combinations(row, 2)
      for pair in pairs:
          error += pair[0] * pair[1]
  return error

def calError_4D(matrix):
  error = 0
  for matrix_3d in matrix:
    for matrix_2d in matrix_3d:
        for row in matrix_2d:
            pairs = itertools.combinations(row, 2)
            for pair in pairs:
                error += pair[0] * pair[1]
  return error

def calculateIPF(file_name, dataInterval, stopAtIndex):

  mainDF = pd.read_csv(file_name, header = 0)

  numOfRows = len(mainDF)
  norm = numOfRows*(numOfRows-1)

  # last column in dataset is the Z attribute
  A_name_list = mainDF.columns[:-1].tolist()

  mainIPFDict = {}
  boundsTimeIPF2D = {}
  boundsTimeIPF3D = {}
  boundsTimeIPF4D = {}

  # THREE-LHS AFDs

  attribute_triplets = list(combinations(mainDF.columns[:-1], 3))

  for attr1, attr2, attr3 in attribute_triplets:

    A_name = attr1+attr2+attr3

    attr1_values = mainDF[attr1].unique().tolist()
    attr2_values = mainDF[attr2].unique().tolist()
    attr3_values = mainDF[attr3].unique().tolist()
    target_values = mainDF['y'].unique().tolist()

    matrix_3lhs = [[[[0 for _ in target_values] for _ in attr3_values] for _ in attr2_values] for _ in attr1_values]

    attr1_indices_map = {value: idx for idx, value in enumerate(attr1_values)}
    attr2_indices_map = {value: idx for idx, value in enumerate(attr2_values)}
    attr3_indices_map = {value: idx for idx, value in enumerate(attr3_values)}
    target_indices_map = {value: idx for idx, value in enumerate(target_values)}

    rowSum_a1 = [0]*len(attr1_indices_map)
    rowSum_a2 = [0]*len(attr2_indices_map)
    rowSum_a3 = [0]*len(attr3_indices_map)
    colSum = [0]*len(target_indices_map)

    for index, row in mainDF.iterrows():
      a1_index = attr1_indices_map[row[attr1]]
      a2_index = attr2_indices_map[row[attr2]]
      a3_index = attr3_indices_map[row[attr3]]
      z_index = target_indices_map[row['y']]

      rowSum_a1[a1_index] += 1
      rowSum_a2[a2_index] += 1
      rowSum_a3[a3_index] += 1
      colSum[z_index] += 1

    dimensions_4D = [[0], [1], [2], [3]]
    constraints_4D = [rowSum_a1, rowSum_a2, rowSum_a3, colSum]

    for index, row in mainDF.iterrows():

      if index > stopAtIndex:
        break

      val1 = row[attr1]
      val2 = row[attr2]
      val3 = row[attr3]
      target_val = row['y']

      if val1 in attr1_indices_map and val2 in attr2_indices_map and val3 in attr3_indices_map and target_val in target_indices_map:

        row_index = attr1_indices_map[val1]
        col_index = attr2_indices_map[val2]
        depth_index = attr3_indices_map[val3]
        target_index = target_indices_map[target_val]

        matrix_3lhs[row_index][col_index][depth_index][target_index] += 1

        if index % dataInterval == 0 or index == len(mainDF)-1:

          if index not in mainIPFDict:
            mainIPFDict[index] = {}

          matrix_3lhs_np = np.array(matrix_3lhs)
          startTimeIPF4D = time.time()
          ipfMatrix_4D = ipfn.ipfn(matrix_3lhs_np, constraints_4D, dimensions_4D)
          matrix_3lhs_ipf = np.round(ipfMatrix_4D.iteration()).astype(int)
          error = calError_4D(matrix_3lhs_ipf)
          stopTimeIPF4D = time.time()

          boundsTimeIPF4D[index] = stopTimeIPF4D - startTimeIPF4D
          mainIPFDict[index][str(A_name)] = error/norm

  # THREE-LHS AFDs

  # TWO-LHS AFDs

  attribute_pairs = list(combinations(mainDF.columns[:-1], 2))

  for attr1, attr2 in attribute_pairs:

    A_name = attr1+attr2
    attr1_values = mainDF[attr1].unique().tolist()
    attr2_values = mainDF[attr2].unique().tolist()
    target_values = mainDF['y'].unique().tolist()

    matrix_2lhs = [[[0 for _ in target_values] for _ in attr2_values] for _ in attr1_values]

    attr1_indices_map = {value: idx for idx, value in enumerate(attr1_values)}
    attr2_indices_map = {value: idx for idx, value in enumerate(attr2_values)}
    target_indices_map = {value: idx for idx, value in enumerate(target_values)}

    rowSum_a1 = [0]*len(attr1_indices_map)
    rowSum_a2 = [0]*len(attr2_indices_map)
    colSum = [0]*len(target_indices_map)

    for index, row in mainDF.iterrows():
      a1_index = attr1_indices_map[row[attr1]]
      a2_index = attr2_indices_map[row[attr2]]
      z_index = target_indices_map[row['y']]

      rowSum_a1[a1_index] += 1
      rowSum_a2[a2_index] += 1
      colSum[z_index] += 1

    dimensions_3D = [[0], [1], [2]]
    constraints_3D = [rowSum_a1, rowSum_a2, colSum]

    for index, row in mainDF.iterrows():

      if index > stopAtIndex:
        break

      val1 = row[attr1]
      val2 = row[attr2]
      target_val = row['y']

      if val1 in attr1_indices_map and val2 in attr2_indices_map and target_val in target_indices_map:

        row_index = attr1_indices_map[val1]
        col_index = attr2_indices_map[val2]
        target_index = target_indices_map[target_val]

        matrix_2lhs[row_index][col_index][target_index] += 1

        if index % dataInterval == 0 or index == len(mainDF)-1:

          matrix_2lhs_np = np.array(matrix_2lhs)
          startTimeIPF3D = time.time()
          ipfMatrix_3D = ipfn.ipfn(matrix_2lhs_np, constraints_3D, dimensions_3D)
          matrix_2lhs_ipf = np.round(ipfMatrix_3D.iteration()).astype(int)
          error = calError_3D(matrix_2lhs_ipf)
          stopTimeIPF3D = time.time()

          boundsTimeIPF3D[index] = stopTimeIPF3D - startTimeIPF3D
          mainIPFDict[index][str(A_name)] = error/norm

  # TWO-LHS AFDs

  # ONE-LHS AFDs

  for A_name in A_name_list:

    Z_name = "y"
    df = mainDF[[A_name, Z_name]]

    a_indices = np.unique(df[A_name])
    a_indices = a_indices.tolist()
    a_indices_map = dict(zip(a_indices, range(len(a_indices))))

    z_indices = np.unique(df[Z_name])
    z_indices = z_indices.tolist()
    z_indices_map = dict(zip(z_indices, range(len(z_indices))))

    dataset = df.values.tolist()
    row_sum = [0]*len(a_indices)
    col_sum = [0]*len(z_indices)

    for data in dataset:
      row_index = a_indices_map[data[0]]
      col_index = z_indices_map[data[1]]
      row_sum[row_index] += 1
      col_sum[col_index] += 1

    calc_matrix = []
    dimensions_2D = [[0], [1]]
    constraints_2D = [row_sum, col_sum]

    for i in a_indices:
      calc_matrix.append([0]*len(z_indices))

    for index,data in enumerate(dataset):

      if index > stopAtIndex:
        break

      row_index = a_indices_map[data[0]]
      col_index = z_indices_map[data[1]]
      calc_matrix[row_index][col_index] += 1

      if index % dataInterval == 0 or index == len(dataset)-1:

        calc_matrix_np = np.array(calc_matrix)
        startTimeIPF2D = time.time()
        ipfMatrix_2D = ipfn.ipfn(calc_matrix_np, constraints_2D, dimensions_2D)
        calc_matrix_ipf = np.round(ipfMatrix_2D.iteration()).astype(int)
        error = calError_2D(calc_matrix_ipf)
        stopTimeIPF2D = time.time()

        boundsTimeIPF2D[index] = stopTimeIPF2D - startTimeIPF2D
        mainIPFDict[index][str(A_name)] = error/norm

  # ONE-LHS AFDs

  indexTotalTimeDictIPF = {key: boundsTimeIPF2D.get(key, 0) + boundsTimeIPF3D.get(key, 0) + boundsTimeIPF4D.get(key, 0)
               for key in set(boundsTimeIPF2D) | set(boundsTimeIPF3D) | set(boundsTimeIPF4D)}

  return mainIPFDict, indexTotalTimeDictIPF

**TANE/Greedy - Baseline**

In [ ]:
import numpy as np
import pandas as pd

def calculateBaseline(file_name, dataInterval, stopAtIndex):

  mainDF = pd.read_csv(file_name, header = 0)

  numOfRows = len(mainDF)
  norm = numOfRows*(numOfRows-1)

  # last column in dataset is the Z attribute
  A_name_list = mainDF.columns[:-1].tolist()

  mainBLDict = {}

  # THREE-LHS AFDs

  attribute_triplets = list(combinations(mainDF.columns[:-1], 3))

  for attr1, attr2, attr3 in attribute_triplets:

    A_name = attr1+attr2+attr3

    attr1_values = mainDF[attr1].unique().tolist()
    attr2_values = mainDF[attr2].unique().tolist()
    attr3_values = mainDF[attr3].unique().tolist()
    target_values = mainDF['y'].unique().tolist()

    matrix_3lhs = [[[[0 for _ in target_values] for _ in attr3_values] for _ in attr2_values] for _ in attr1_values]

    attr1_indices_map = {value: idx for idx, value in enumerate(attr1_values)}
    attr2_indices_map = {value: idx for idx, value in enumerate(attr2_values)}
    attr3_indices_map = {value: idx for idx, value in enumerate(attr3_values)}
    target_indices_map = {value: idx for idx, value in enumerate(target_values)}

    rowSum_a1 = [0]*len(attr1_indices_map)
    rowSum_a2 = [0]*len(attr2_indices_map)
    rowSum_a3 = [0]*len(attr3_indices_map)
    colSum = [0]*len(target_indices_map)

    for index, row in mainDF.iterrows():
      a1_index = attr1_indices_map[row[attr1]]
      a2_index = attr2_indices_map[row[attr2]]
      a3_index = attr3_indices_map[row[attr3]]
      z_index = target_indices_map[row['y']]

      rowSum_a1[a1_index] += 1
      rowSum_a2[a2_index] += 1
      rowSum_a3[a3_index] += 1
      colSum[z_index] += 1

    for index, row in mainDF.iterrows():

      if index > stopAtIndex:
        break

      val1 = row[attr1]
      val2 = row[attr2]
      val3 = row[attr3]
      target_val = row['y']

      if val1 in attr1_indices_map and val2 in attr2_indices_map and val3 in attr3_indices_map and target_val in target_indices_map:

        row_index = attr1_indices_map[val1]
        col_index = attr2_indices_map[val2]
        depth_index = attr3_indices_map[val3]
        target_index = target_indices_map[target_val]

        matrix_3lhs[row_index][col_index][depth_index][target_index] += 1

        if index % dataInterval == 0 or index == len(mainDF)-1:

          if index not in mainBLDict:
            mainBLDict[index] = {}

          g1ErrorBL_4D = 0

          for matrix_3d in matrix_3lhs:
            for matrix_2d in matrix_3d:
                for row in matrix_2d:
                    pairs = itertools.combinations(row, 2)
                    for pair in pairs:
                        g1ErrorBL_4D += pair[0] * pair[1]

          mainBLDict[index][str(A_name)] = g1ErrorBL_4D/norm

  # THREE-LHS AFDs

  # TWO-LHS AFDs

  attribute_pairs = list(combinations(mainDF.columns[:-1], 2))

  for attr1, attr2 in attribute_pairs:

    A_name = attr1+attr2
    attr1_values = mainDF[attr1].unique().tolist()
    attr2_values = mainDF[attr2].unique().tolist()
    target_values = mainDF['y'].unique().tolist()

    matrix_2lhs = [[[0 for _ in target_values] for _ in attr2_values] for _ in attr1_values]

    attr1_indices_map = {value: idx for idx, value in enumerate(attr1_values)}
    attr2_indices_map = {value: idx for idx, value in enumerate(attr2_values)}
    target_indices_map = {value: idx for idx, value in enumerate(target_values)}

    rowSum_a1 = [0]*len(attr1_indices_map)
    rowSum_a2 = [0]*len(attr2_indices_map)
    colSum = [0]*len(target_indices_map)

    for index, row in mainDF.iterrows():
      a1_index = attr1_indices_map[row[attr1]]
      a2_index = attr2_indices_map[row[attr2]]
      z_index = target_indices_map[row['y']]

      rowSum_a1[a1_index] += 1
      rowSum_a2[a2_index] += 1
      colSum[z_index] += 1

    for index, row in mainDF.iterrows():

      if index > stopAtIndex:
        break

      val1 = row[attr1]
      val2 = row[attr2]
      target_val = row['y']

      if val1 in attr1_indices_map and val2 in attr2_indices_map and target_val in target_indices_map:

        row_index = attr1_indices_map[val1]
        col_index = attr2_indices_map[val2]
        target_index = target_indices_map[target_val]

        matrix_2lhs[row_index][col_index][target_index] += 1

        if index % dataInterval == 0 or index == len(mainDF)-1:

          g1ErrorBL_3D = 0

          for matrix_2d in matrix_2lhs:
            for row in matrix_2d:
                pairs = itertools.combinations(row, 2)
                for pair in pairs:
                    g1ErrorBL_3D += pair[0] * pair[1]

          mainBLDict[index][str(A_name)] = g1ErrorBL_3D/norm

  # TWO-LHS AFDs

  # ONE-LHS AFDs

  for A_name in A_name_list:

    Z_name = "y"
    df = mainDF[[A_name, Z_name]]

    a_indices = np.unique(df[A_name])
    a_indices = a_indices.tolist()
    a_indices_map = dict(zip(a_indices, range(len(a_indices))))

    z_indices = np.unique(df[Z_name])
    z_indices = z_indices.tolist()
    z_indices_map = dict(zip(z_indices, range(len(z_indices))))

    row_sum = [0]*len(a_indices)
    col_sum = [0]*len(z_indices)

    matrix = []
    for i in a_indices:
      matrix.append([0]*len(z_indices))

    dataset = df.values.tolist()

    for index,data in enumerate(dataset):

      if index > stopAtIndex:
        break

      row_index = a_indices_map[data[0]]
      col_index = z_indices_map[data[1]]
      row_sum[row_index] += 1
      col_sum[col_index] += 1
      matrix[row_index][col_index] += 1

      if index % dataInterval == 0 or index == len(dataset)-1:

        g1ErrorBL_2D = 0
        for row in matrix:
          pairs = itertools.combinations(row, 2)
          for pair in pairs:
            g1ErrorBL_2D += pair[0] * pair[1]

        mainBLDict[index][str(A_name)] = g1ErrorBL_2D/norm

  # ONE-LHS AFDs

  return mainBLDict

**Pyro++ - SOTA**

In [ ]:
import re
import os
import pandas as pd
from itertools import combinations

# converting log files generated by Pyro and pandas dataframe to extract AFDs
def log_to_dataframe(log_file):

  data = []

  with open(log_file, 'r') as log:
    for line in log:
      if "LHS:" in line and "RHS:" in line and "Error:" in line:
        try:
          lhs = line.split('LHS:')[1].split('RHS:')[0].strip().strip('[]').replace(']', '').replace(',', '').strip()
          rhs = line.split('RHS:')[1].split('Error:')[0].strip().strip('[]').replace(']', '').replace(',', '').strip()
          error = float(line.split('Error:')[1].strip())

          data.append([lhs, rhs, error])
        except IndexError as e:
          print(f"Skipping line due to error: {e}")
          continue

  sorted_data = sorted(data, key=lambda x: x[2])
  df = pd.DataFrame(sorted_data, columns=['LHS', 'RHS', 'Error'])

  return df

# adapting Pyro++ by generating supersets of minimal AFDs by Pyro
def generate_supersets(sets_str):

  supersets = []
  sets = [set(map(int, s)) for s in sets_str]
  unique_elements = set().union(*sets)

  for r in range(1, 4):
      for combo in combinations(unique_elements, r):
          superset = set(combo)
          if any(s.issubset(superset) for s in sets):
              supersets.append(superset)

  supersets = sorted(supersets, key=lambda x: (len(x), sorted(x)))

  supersets_str = [''.join(sorted(map(str, superset))) for superset in supersets]

  return supersets_str

# score calculation for pyro++
def calculatePyroPatkAndNDCG(filename, actualTopk, actualG1Dict, actualTopk_rs, k):

  df_og = pd.read_csv(filename, header = 0)
  pdbxCol = df_og.columns.tolist()
  numCol = df_og.shape[1]

  pyroplusTopk = []
  patkGraph_pyro = []
  ndcgatkGraph_pyro = []
  folder_path = './' + 'logFiles'

  for fn in sorted(os.listdir(folder_path)):

    # note: only pass files upto the percent of data read (60% for our experiments), pass in correct order

    log_file_path = os.path.join(folder_path, fn)
    df_pyro = log_to_dataframe(log_file_path)
    pyroTopK = []

    for index, row in df_pyro.iterrows():

      col1 = row['LHS']
      col2 = row['RHS']

      if col2[6:] == str(numCol):
        col = re.sub(r'[^0-9]', '', col1)
        pyroTopK.append(col)

    pyroplusDictSorted = {}
    pyroplusDictUnsorted = {}

    if pyroTopK:

      pyroplusList = generate_supersets(pyroTopK)
      pyroplusDictUnsorted = {ky: v for ky, v in actualG1Dict.items() if ky in pyroplusList}
      pyroplusDictSorted = dict(sorted(pyroplusDictUnsorted.items(), key=lambda item: item[1]))
      pyroplusTopk = sorted(pyroplusDictSorted, key=lambda ky: pyroplusDictSorted[ky])

      pyroTopk_rs = adjustRelevanceScore(pyroplusTopk, actualTopk[:k], actualTopk_rs[:k])

      patkGraph_pyro.append(precision_at_k(actualTopk, pyroplusTopk, k=k))
      ndcgatkGraph_pyro.append(ndcg_at_k(pyroTopk_rs, actualTopk_rs, k=k))

    else:
      patkGraph_pyro.append(0.0)
      ndcgatkGraph_pyro.append(0.0)

  return patkGraph_pyro, ndcgatkGraph_pyro

**Score Calculation - P@K/NDCG**

In [ ]:
import numpy as np
import pandas as pd

# Precision at k score
def precision_at_k(y_true, y_pred, k):

  relevantItems = sum([1 for item in y_pred[:k] if item in y_true[:k]])

  return relevantItems/k

# ndcg - true relevance score
def calcRelevanceScore(topkDict, fixedIndexDict, k):

  true_rs = [0] * len(fixedIndexDict)

  if topkDict:
    maxValNorm = topkDict[max(topkDict, key=topkDict.get)]
    for key, value in topkDict.items():
      if maxValNorm != 0:
        normVal_RS = 1-(value/maxValNorm)
      else:
        normVal_RS = 0

      if key in fixedIndexDict:
        true_rs[fixedIndexDict[key]] = normVal_RS

  return true_rs

# ndcg - predicted relevance score
def adjustRelevanceScore(predTopk, trueTopk, true_rs):

  relMap = dict(zip(trueTopk, true_rs))
  predTopk = predTopk[:len(trueTopk)]

  if predTopk:
    pred_rs = [relMap.get(item, 0) for item in predTopk]
    pred_rs += [0] * (len(trueTopk) - len(predTopk))
  else:
    pred_rs = [0] * len(trueTopk)

  return pred_rs

# dcg score
def dcg_at_k(relevance_scores, k):

  dcg = 0.0
  for i in range(min(k, len(relevance_scores))):
      dcg += (2**relevance_scores[i] - 1) / np.log2(i + 2)

  return dcg

# ncdg score
def ndcg_at_k(pred_rel, true_rel, k):

  dcg_max = dcg_at_k(pred_rel, k)
  ideal_relevance_scores = sorted(true_rel, reverse=True)
  idcg = dcg_at_k(ideal_relevance_scores, k)

  return dcg_max/idcg if idcg > 0 else 0

**Generate Results**

In [ ]:
import time
import pandas as pd

# Main Method: makes calls to all algorithms for bound calculation
def calculatePatKandNDCG(file_name, dataInterval, rows, kList, stopAtIndex, fn, sortedStr, e):

  # actual top-k
  actualTopkDict, actualDict = calculateActualTopk(file_name, e)
  actualTopk = sorted(actualTopkDict, key=lambda ky: actualTopkDict[ky])
  fixedIndexAFDDict = {key: idx for idx, key in enumerate(actualTopkDict.keys())}

  # time vs k graphs
  lpdsTimeVsKDict = {}
  ipfTimeVsKDict = {}
  blTimeVsKDict = {}

  for k in kList:

    actualTopk_rs  = calcRelevanceScore(actualTopkDict, fixedIndexAFDDict, k)

    # Algorithm 1 : LPDS
    startKTimeLPDS = time.time()
    patkGraph_lpds, ndcgkGraph_lpds = [], []

    mainUBDict, mainLBDict, lpdsTimeVsDataDict, boundsTime2D, boundsTime3D, boundsTime4D = calculateBoundsLPDS(file_name, dataInterval, stopAtIndex)
    keys = mainUBDict.keys()

    for key in keys:

      ubdataDict = mainUBDict[key]
      lbdataDict = mainLBDict[key]
      attr = list(ubdataDict.keys())

      approxTopk_lpds = cal_mean(attr, ubdataDict, lbdataDict, e)
      approxTopk_rs = adjustRelevanceScore(approxTopk_lpds, actualTopk[:k], actualTopk_rs[:k])

      patkGraph_lpds.append(precision_at_k(actualTopk, approxTopk_lpds, k=k))
      ndcgkGraph_lpds.append(ndcg_at_k(approxTopk_rs, actualTopk_rs, k=k))

    stopKTimeLPDS = time.time()
    lpdsTimeVsKDict[k] = stopKTimeLPDS - startKTimeLPDS

    # Algorithm 2 : IPF
    startKTimeIPF = time.time()
    patkGraph_ipf, ndcgkGraph_ipf = [], []

    mainIPFDict, ipfTimeVsDataDict = calculateIPF(file_name, dataInterval, stopAtIndex)
    keys = mainIPFDict.keys()

    for key in keys:

      ipfDict = mainIPFDict[key]
      attr = list(ipfDict.keys())

      ipfTopkDict = {k: v for k, v in sorted(ipfDict.items(), key=lambda item: item[1]) if v <= e}
      approxTopk_ipf = sorted(ipfTopkDict, key=lambda x: ipfTopkDict[x])
      ipfTopk_rs = adjustRelevanceScore(approxTopk_ipf, actualTopk[:k], actualTopk_rs[:k])

      patkGraph_ipf.append(precision_at_k(actualTopk, approxTopk_ipf, k=k))
      ndcgkGraph_ipf.append(ndcg_at_k(ipfTopk_rs, actualTopk_rs, k=k))

    stopKTimeIPF = time.time()
    ipfTimeVsKDict[k] = stopKTimeIPF - startKTimeIPF

    # Algorithm 3 : Baseline
    startKTimeBL = time.time()
    patkGraph_bl, ndcgkGraph_bl = [], []

    mainBLDict = calculateBaseline(file_name, dataInterval, stopAtIndex)
    keys = mainBLDict.keys()

    for key in keys:

      blDict = mainBLDict[key]
      attr = list(blDict.keys())

      blTopkDict = {k: v for k, v in sorted(blDict.items(), key=lambda item: item[1]) if v <= e}
      approxTopk_bl = sorted(blTopkDict, key=lambda y: blTopkDict[y])
      blTopk_rs = adjustRelevanceScore(approxTopk_bl, actualTopk[:k], actualTopk_rs[:k])

      patkGraph_bl.append(precision_at_k(actualTopk, approxTopk_bl, k=k))
      ndcgkGraph_bl.append(ndcg_at_k(blTopk_rs, actualTopk_rs, k=k))

    stopKTimeBL = time.time()
    blTimeVsKDict[k] = stopKTimeBL - startKTimeBL

    # Algorithm 4 : Pyro++
    patkGraph_pyro, ndcgkGraph_pyro = [], []
    patkGraph_pyro, ndcgkGraph_pyro = calculatePyroPatkAndNDCG(file_name, actualTopk, actualDict, actualTopk_rs, k)

    # p@k and ndcg graphs
    generatePatK(patkGraph_lpds, patkGraph_ipf, patkGraph_bl, patkGraph_pyro, rows, dataInterval, k, stopAtIndex, fn, sortedStr)
    generateNDCGK(ndcgkGraph_lpds, ndcgkGraph_ipf, ndcgkGraph_bl, ndcgkGraph_pyro, rows, dataInterval, k, stopAtIndex, fn, sortedStr)

  # clocktime graphs
  generateKClocktimeGraph(lpdsTimeVsKDict, ipfTimeVsKDict, blTimeVsKDict, kList, fn, sortedStr)
  generateDataClocktimeGraph(lpdsTimeVsDataDict, ipfTimeVsDataDict, rows, dataInterval, stopAtIndex, fn, sortedStr)
  generateLPDSBoundsClocktimeGraph(boundsTime2D, boundsTime3D, boundsTime4D, rows, dataInterval, stopAtIndex, fn, sortedStr)

  return None

**Precision at k Graph**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def generatePatK(patkGraph_lpds, patkGraph_ipf, patkGraph_bl, patkGraph_pyro, rows, dataInterval, k, stopAtIndex, fn, sortedStr):

  rows_read = list(range(0, round(stopAtIndex) + 1, dataInterval))
  rows_read = [round((x / rows) * 100) for x in rows_read]

  bar_width = 0.15
  index = np.arange(len(rows_read))

  patterns = ['\\', '.', '-', 'x']

  plt.bar(index, patkGraph_lpds, bar_width, label='AppxLP', hatch=patterns[0], facecolor='skyblue', edgecolor='black')
  plt.bar(index + bar_width, patkGraph_ipf, bar_width, label='AppxIPF', hatch=patterns[1], facecolor='khaki', edgecolor='black')
  plt.bar(index + 2 * bar_width, patkGraph_bl, bar_width, label='TANE', hatch=patterns[2], facecolor='darkseagreen', edgecolor='black')
  plt.bar(index + 3 * bar_width, patkGraph_pyro, bar_width, label='Pyro++', hatch=patterns[3], facecolor='lightsalmon', edgecolor='black')

  plt.xlabel('Percent of Data Read', fontsize=16, fontweight='bold')
  plt.ylabel('Precision', fontsize=16, fontweight='bold')

  plt.xticks(index + bar_width * 1.75, [f'{round(r, 0)}' for r in rows_read], fontsize=16)
  plt.yticks(fontsize=16)

  plt.grid(True, linestyle='--', linewidth=0.5)
  plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.22), ncol=2, prop={'weight': 'bold', 'size': 14})

  plt.savefig('P@K=' + str(k) + '_' + fn + '_' + sortedStr + '.png', dpi=300, bbox_inches='tight')
  plt.close()

**NDCG Graph**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def generateNDCGK(ndcgkGraph_lpds, ndcgkGraph_ipf, ndcgkGraph_bl, ndcgkGraph_pyro, rows, dataInterval, k, stopAtIndex, fn, sortedStr):

  rows_read = list(range(0, round(stopAtIndex) + 1, dataInterval))
  rows_read = [round((x / rows) * 100) for x in rows_read]

  bar_width = 0.15
  index = np.arange(len(rows_read))

  patterns = ['\\', '.', '-', 'x']

  plt.bar(index, ndcgkGraph_lpds, bar_width, label='AppxLP', hatch=patterns[0], facecolor='skyblue', edgecolor='black')
  plt.bar(index + bar_width, ndcgkGraph_ipf, bar_width, label='AppxIPF', hatch=patterns[1], facecolor='khaki', edgecolor='black')
  plt.bar(index + 2 * bar_width, ndcgkGraph_bl, bar_width, label='TANE', hatch=patterns[2], facecolor='darkseagreen', edgecolor='black')
  plt.bar(index + 3 * bar_width, ndcgkGraph_pyro, bar_width, label='Pyro++', hatch=patterns[3], facecolor='lightsalmon', edgecolor='black')

  plt.xlabel('Percent of Data Read', fontsize=16, fontweight='bold')
  plt.ylabel('NDCG', fontsize=16, fontweight='bold')

  plt.xticks(index + bar_width * 1.75, [f'{round(r, 0)}' for r in rows_read], fontsize=16)
  plt.yticks(fontsize=16)

  plt.grid(True, linestyle='--', linewidth=0.5)
  plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.22), ncol=2, prop={'weight': 'bold', 'size': 14})

  plt.savefig('NDCG@K=' + str(k) + '_' + fn + '_' + sortedStr + '.png', dpi=300, bbox_inches='tight')
  plt.close()

**K vs Clocktime Graph**

In [ ]:
import matplotlib.pyplot as plt

def generateKClocktimeGraph(lpdsTimeVsKDict, ipfTimeVsKDict, blTimeVsKDict, kList, fn, sortedStr):

  x = kList
  y1 = list(lpdsTimeVsKDict.values())
  y2 = list(ipfTimeVsKDict.values())

  plt.figure(figsize=(14, 8))
  plt.plot(x, y1, marker='o', linestyle='-', color='dodgerblue', label='AppxLP', linewidth=5)
  plt.plot(x, y2, marker='x', linestyle='-', color='gold', label='AppxIPF', linewidth=5)

  plt.xticks(x, fontsize=35)
  y_min = min(min(y1), min(y2)) - 0.05
  y_max = max(max(y1), max(y2)) + 0.05
  yticks = np.round(np.linspace(y_min, y_max, num=5), 3)

  plt.yticks(yticks, fontsize=35)

  plt.xlabel('k', fontsize=34, fontweight='bold')
  plt.ylabel('Time (sec)', fontsize=34, fontweight='bold')

  plt.grid(True, linestyle='--', linewidth=0.5)
  plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.20), ncol=2, prop={'weight': 'bold', 'size': 35})

  plt.savefig('AppxKvsClocktime'+'_'+fn+'_'+sortedStr+'.png', dpi=300, bbox_inches='tight')
  plt.close()

**Data Read vs Clocktime Graph**

In [ ]:
import matplotlib.pyplot as plt

def generateDataClocktimeGraph(lpdsTimeVsDataDict, ipfTimeVsDataDict, rows, dataInterval, stopAtIndex, fn, sortedStr):

  rows_read = list(range(0, round(stopAtIndex) + 1, dataInterval))
  rows_read = [round((x / rows) * 100) for x in rows_read]

  y1 = list(lpdsTimeVsDataDict.values())
  y2 = list(ipfTimeVsDataDict.values())

  plt.figure(figsize=(14, 8))
  plt.plot(rows_read, y1, marker='o', linestyle='-', color='dodgerblue', label='AppxLP', linewidth=5)
  plt.plot(rows_read, y2, marker='x', linestyle='-', color='gold', label='AppxIPF', linewidth=5)

  plt.xticks(rows_read, fontsize=35)
  y_min = min(min(y1), min(y2)) - 0.001
  y_max = max(max(y1), max(y2)) + 0.001
  yticks = np.round(np.linspace(y_min, y_max, num=6), 4)
  plt.yticks(yticks, fontsize=35)

  plt.xlabel('Data Read', fontsize=34, fontweight='bold')
  plt.ylabel('Time (sec)', fontsize=34, fontweight='bold')

  plt.grid(True, linestyle='--', linewidth=0.5)
  plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.20), ncol=2, prop={'weight': 'bold', 'size': 35})

  plt.savefig('AppxDatavsClocktime'+'_'+fn+'_'+sortedStr+'.png', dpi=300, bbox_inches='tight')
  plt.close()

**Bounds vs Clocktime Graph**

In [ ]:
import matplotlib.pyplot as plt

def generateLPDSBoundsClocktimeGraph(boundsTime2D, boundsTime3D, boundsTime4D, rows, dataInterval, stopAtIndex, fn, sortedStr):

  rows_read = list(range(0, round(stopAtIndex) + 1, dataInterval))
  rows_read = [round((x / rows) * 100) for x in rows_read]

  bounds2D = list(boundsTime2D.values())
  bounds3D = list(boundsTime3D.values())
  bounds4D = list(boundsTime4D.values())

  plt.figure(figsize=(14, 8))
  plt.plot(rows_read, bounds2D, marker='^', linestyle='-', color='black', label='One LHS AFD')
  plt.plot(rows_read, bounds3D, marker='D', linestyle='-', color='black', label='Two LHS AFD')
  plt.plot(rows_read, bounds4D, marker='s', linestyle='-', color='black', label='Three LHS AFD')

  plt.xticks(fontsize=14)
  plt.yticks(fontsize=14)

  plt.xlabel('Data Read', fontsize=20, fontweight='bold')
  plt.ylabel('Time (sec)', fontsize=20, fontweight='bold')

  plt.grid(True, linestyle='--', linewidth=0.5)
  plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=2, prop={'weight': 'bold', 'size': 20})

  plt.savefig('AppxBoundsvsClocktime'+'_'+fn+'_'+sortedStr+'.png', dpi=300, bbox_inches='tight')
  plt.close()

**Dataset Sorting (Key/Non-Key)**

In [ ]:
import pandas as pd

def sortDataset(file_name, attrList, attr, sortOrder):

  if attr != '0':
    df = pd.read_csv(file_name)
    df_sorted = df.sort_values(by=attrList, ascending=sortOrder)
    sortedFN = 'Sorted_X'+attr+'_'+file_name
    df_sorted.to_csv(sortedFN, index=False)
  else:
    return file_name

  return sortedFN

**Start Point** (Define parameters and run code)

In [ ]:
def getDatasetParam(datasetName):

    dataset_config = {
        'dataset1': {'fn': 'DB Status', 'rows': 10000, 'interval': 1000, 'fileName': 'DBStatus.csv'},
        'dataset2': {'fn': 'Letter', 'rows': 20000, 'interval': 2000, 'fileName': 'Letter.csv'},
        'dataset3': {'fn': 'Student', 'rows': 1000, 'interval': 100, 'fileName': 'Student.csv'},
        'dataset4': {'fn': 'Synthetic1', 'rows': 100000, 'interval': 10000, 'fileName': 'SyntheticData1.csv'},
        'dataset5': {'fn': 'Synthetic2', 'rows': 100000, 'interval': 10000, 'fileName': 'SyntheticData2.csv'},
        'dataset6': {'fn': 'Synthetic3', 'rows': 160000, 'interval': 16000, 'fileName': 'SyntheticData3.csv'},
        'dataset7': {'fn': 'Synthetic4', 'rows': 48000, 'interval': 4800, 'fileName': 'SyntheticData4.csv'}
    }

    if datasetName in dataset_config:
        return dataset_config[datasetName]
    else:
        raise ValueError(f"{datasetName} not found.")

def AFDStartPoint():

  # define dataset paramters
  datasetNum = 'dataset1'
  params = getDatasetParam(datasetNum)
  fn = params['fn']
  rows = params['rows']
  interval = params['interval']
  fileName = params['fileName']

  # define sorting order
  attr = '0'
  attrList = ['0']
  sortOrder = [True]
  sortedStr = ''
  data = sortDataset(fileName, attrList, attr, sortOrder)

  # threshold used by pyro
  e = 0.01
  # define k values
  kList = [3,5,7]
  # define limit of records read
  stopAtIndex = 0.6*rows

  # pass the parameters and generate results
  calculatePatKandNDCG(data, interval, rows, kList, stopAtIndex, fn, sortedStr, e)

In [ ]:
# start point
AFDStartPoint()

4D Actual Conflict {'123': 0.009844234423442344, '124': 0.010122462246224623, '125': 0.010172777277727772, '126': 0.010645844584458447, '134': 0.009526092609260926, '135': 0.009858185818581859, '136': 0.009692269226922692, '145': 0.009831793179317932, '146': 0.009534183418341835, '156': 0.009887938793879388, '234': 0.009508060806080607, '235': 0.00984015401540154, '236': 0.00968927892789279, '245': 0.009777697769776978, '246': 0.009511591159115911, '256': 0.009865406540654065, '345': 0.009522912291229122, '346': 0.009368646864686468, '356': 0.009691269126912691, '456': 0.009513861386138614}
3D Actual Conflict {'12': 0.02242954295429543, '13': 0.009862266226622663, '14': 0.010212621262126213, '15': 0.010226872687268728, '16': 0.010670737073707371, '23': 0.009844234423442344, '24': 0.010122462246224623, '25': 0.010172777277727772, '26': 0.010645844584458447, '34': 0.009526092609260926, '35': 0.009858185818581859, '36': 0.009692269226922692, '45': 0.009831793179317932, '46': 0.00953418341